In [28]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [29]:
data = pd.read_csv('resume_data.csv', encoding='utf-8')
data

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,NaN,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,NaN,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...",...,NaN,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['Company Name', 'Company Name', 'Company Name...",...,[None],[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,NaN,NaN,"['Mathematical modelling', 'Machine Learning',...",['Sanghvi College of Engineering'],['B.Tech'],['2019'],['N/A'],['N/A'],['N/A'],['BPM Foundation'],...,NaN,NaN,NaN,Data Engineer,Bachelor of Science (BSc),5 to 8 years,NaN,Data Platform Design\nData Pipeline Developmen...,Azure\nBig Data\nData Analytics\nETL Tools\nPo...,0.683333
9540,NaN,Expertise EDA modeler. I like to learn what my...,"['Data Analysis', 'Business Analysis', 'Machin...","['KVoCT, Pune', 'KVoCT, Pune']","['B.CA', 'M.CA']","['2018', '2020']","[None, None]","[None, None]","[None, None]",['Passionate Solution'],...,NaN,NaN,NaN,Executive/ Sr. Executive -IT,Bachelor of Science (BSc) in Computer Science ...,3 to 5 years,Age at most 40 years,Hardware & Software Installation\nSystem Monit...,NaN,0.650000
9541,NaN,Looking for roles related to application devel...,"['Business Analyst', 'Data Analytics', 'Data C...",['PGG College Mysore'],['B.BA'],['2019'],['N/A'],['N/A'],['N/A'],['ZigSAW'],...,NaN,NaN,NaN,Executive - VAT,BBA in Accounting and Finance,1 to 3 years,NaN,Mushak Forms Maintenance\nVAT Software & MS Of...,VAT and Tax,0.650000
9542,NaN,NaN,"['Machine Learning', 'Natural Language Process...","['Rajiv Gandhi Me

In [42]:
# Hàm tiền xử lý văn bản
def clean_skills(text):
    if pd.isna(text): return ""  # Xử lý giá trị NaN
    text = text.lower()  # Chuyển thành chữ thường
    return text

# Áp dụng tiền xử lý lên cột title và description
data['skills'] = data['skills'].apply(clean_skills)
data['related_skils_in_job'] = data['related_skils_in_job'].apply(clean_skills)

In [44]:
# Từ khóa IT để gán nhãn
it_keywords = ['python', 'java', 'c++', 'c#', 'javascript', 'typescript', 'html', 'css', 'react', 'angular',
    'vue', 'nodejs', 'express', 'django', 'flask', 'sql', 'mysql', 'postgresql', 'mongodb',
    'git', 'docker', 'kubernetes', 'aws', 'azure', 'gcp', 'tensorflow', 'pytorch', 'machine learning',
    'deep learning', 'ai', 'nlp', 'computer vision', 'data science', 'big data', 'hadoop', 'spark',
    'linux', 'bash', 'shell scripting', 'api', 'rest', 'graphql', 'cybersecurity', 'penetration testing',
    'blockchain', 'smart contracts', 'devops', 'agile', 'scrum', 'software development', 'backend',
    'frontend', 'fullstack', 'data analysis', 'cloud computing', 'networking', 'system administration']

# Hàm gán nhãn IT (1) hoặc Non-IT (0)
def label_it_job(text):
    if isinstance(text, str):  # Kiểm tra nếu text là chuỗi
        return 1 if any(keyword in text.lower() for keyword in it_keywords) else 0
    return 0  # Nếu không phải chuỗi, gán mặc định là 0

# Gán nhãn vào cột mới
data['is_it'] = data['skills'].apply(label_it_job)

# Kiểm tra số lượng bài đăng IT và Non-IT
print(data['is_it'].value_counts())
data

is_it
1    7920
0    1624
Name: count, dtype: int64


,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score,is_it,is_it_predicted
0,NaN,Big data analytics working and database wareho...,"['big data', 'hadoop', 'hive', 'python', 'mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000,1,1
1,NaN,Fresher looking to join as a data analyst and ...,"['data analysis', 'data analytics', 'business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000,1,1
2,NaN,NaN,"['software development', 'machine learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667,1,1
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...",...,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000,1,1
4,NaN,Professional accountant with an outstanding wo...,"['analytical reasoning', 'compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['Company Name', 'Company Name', 'Company Name...",...,"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,NaN,NaN,"['mathematical modelling', 'machine learning',...",['Sanghvi College of Engineering'],['B.Tech'],['2019'],['N/A'],['N/A'],['N/A'],['BPM Foundation'],...,NaN,Data Engineer,Bachelor of Science (BSc),5 to 8 years,NaN,Data Platform Design\nData Pipeline Developmen...,Azure\nBig Data\nData Analytics\nETL Tools\nPo...,0.683333,1,1
9540,NaN,Expertise EDA modeler. I like to learn what my...,"['data analysis', 'business analysis', 'machin...","['KVoCT, Pune', 'KVoCT, Pune']","['B.CA', 'M.CA']","['2018', '2020']","[None, None]","[None, None]","[None, None]",['Passionate Solution'],...,NaN,Executive/ Sr. Executive -IT,Bachelor of Science (BSc) in Computer Science ...,3 to 5 years,Age at most 40 years,Hardware & Software Installation\nSystem Monit...,NaN,0.650000,1,1
9541,NaN,Looking for roles related to application devel...,"['business analyst', 'data analytics', 'data c...",['PGG College Mysore'],['B.BA'],['2019'],['N/A'],['N/A'],['N/A'],['ZigSAW'],...,NaN,Executive - VAT,BBA in Accounting and Finance,1 to 3 years,NaN,Mushak Forms Maintenance\nVAT Software & MS Of...,VAT and Tax,0.650000,1,1
9542,NaN,NaN,"['machine learning', 'natural language process...","['Rajiv Gandhi Memorial University, Delhi']",['B.TECH'],['

In [45]:

data['skills'] = data['skills'].fillna("")
data['related_skils_in_job'] = data['related_skils_in_job'].fillna("")

# Kết hợp hai cột thành một để huấn luyện mô hình
data['combined_skills'] = data['skills'] + " " + data['related_skils_in_job']

# Sử dụng TF-IDF để trích xuất đặc trưng từ cả hai cột
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['combined_skills'])

# Lấy nhãn
y = data['is_it']
            

In [46]:
# Chia dữ liệu thành tập huấn luyện (80%) và kiểm tra (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo mô hình Logistic Regression
model = LogisticRegression()
# model = RandomForestClassifier(n_estimators=100)  # Thử Random Forest nếu muốn

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

In [47]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9848088004190676
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.91      0.95       336
           1       0.98      1.00      0.99      1573

    accuracy                           0.98      1909
   macro avg       0.99      0.96      0.97      1909
weighted avg       0.99      0.98      0.98      1909



In [38]:
# Dự đoán trên toàn bộ dữ liệu
data['is_it_predicted'] = model.predict(X)

# Lọc các bài đăng IT
data_it = data[data['is_it_predicted'] == 1]

In [39]:
# Xuất file mới chứa danh sách việc làm IT
data_it.to_csv("filtered_it_skills.csv", index=False, encoding='utf-8')
print("Lưu danh sách việc làm IT vào filtered_it_cv.csv")

Lưu danh sách việc làm IT vào filtered_it_cv.csv


In [10]:
skill_data = pd.read_csv('filtered_it_cv.csv', encoding='utf-8')

In [11]:
skill_data['skills'] = skill_data['skills'].str.split(r'\s*[,;.]\s*')
skill_data = skill_data.explode('skills')
skill_data

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score,is_it,is_it_predicted
0,NaN,big data analytics working and database wareho...,['Big Data',['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.85,1,1
0,NaN,big data analytics working and database wareho...,'Hadoop',['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.85,1,1
0,NaN,big data analytics working and database wareho...,'Hive',['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.85,1,1
0,NaN,big data analytics working and database wareho...,'Python',['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.85,1,1
0,NaN,big data analytics working and database wareho...,'Mapreduce',['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.85,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,NaN,looking for roles related to application devel...,'Statistical Analysis',['PGG College Mysore'],['B.BA'],['2019'],['N/A'],['N/A'],['N/A'],['ZigSAW'],...,NaN,Executive - VAT,BBA in Accounting and Finance,1 to 3 years,NaN,Mushak Forms Maintenance\nVAT Software & MS Of...,VAT and Tax,0.65,1,1
3247,NaN,looking for roles related to application devel...,'Deep Learning',['PGG College Mysore'],['B.BA'],['2019'],['N/A'],['N/A'],['N/A'],['ZigSAW'],...,NaN,Executive - VAT,BBA in Accounting and Finance,1 to 3 years,NaN,Mushak Forms Maintenance\nVAT Software & MS Of...,VAT and Tax,0.65,1,1
3247,NaN,looking for roles related to application devel...,'Machine Learning',['PGG College Mysore'],['B.BA'],['2019'],['N/A'],['N/A'],['N/A'],['ZigSAW'],...,NaN,Executive - VAT,BBA in Accounting and Finance,1 to 3 years,NaN,Mushak Forms Maintenance\nVAT Software & MS Of...,VAT and Tax,0.65,1,1
3247,NaN,looking for roles related to application devel...,'Python',['PGG College Mysore'],['B.BA'],['2019'],['N/A'],['N/A'],['N/A'],['ZigSAW'],...,NaN,Executive - VAT,BBA in Accounting and Finance,1 to 3 years,NaN,Mushak Forms Maintenance\nVAT Software & MS Of...,VAT and Tax,0.65,1,1


In [12]:
def contains_it_keywords(desc):
    return any(keyword in desc for keyword in it_keywords)
skill_data = skill_data[skill_data['skills'].apply(contains_it_keywords)]
skill_data

TypeError: argument of type 'float' is not iterable

In [ ]:
# Hàm làm sạch: Xóa các ký tự đặc biệt, chỉ giữ lại chữ cái, số và khoảng trắng
def clean_skills(text):
    if isinstance(text, str):
        return re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Xóa mọi thứ trừ chữ cái, số và khoảng trắng
    return text  # Nếu không phải chuỗi, giữ nguyên

# Áp dụng hàm làm sạch vào cột 'skills'
skill_data['skills'] = skill_data['skills'].apply(clean_skills)

C:\Users\trant\AppData\Local\Temp\ipykernel_8548\1732856744.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_data['description'] = skill_data['description'].apply(clean_description)


In [ ]:
choosen_column = [
    'job_id',
    'company_name',
    'title',
    'skills',
    'max_salary',
]
skill_data = skill_data[choosen_column]
skill_data

,job_id,company_name,title,description,max_salary
0,133130219,NaN,software engineer,Education Bachelors degree in software,NaN
1,175485704,GOYT,software engineer,ResponsibilitiesDevelop and maintain highquali...,NaN
1,175485704,GOYT,software engineer,Familiarity with MySQL or other relational dat...,NaN
1,175485704,GOYT,software engineer,If you are a talented PHP developer with a pas...,NaN
2,266825034,Recruitment Design,software support specialist,but are not limited to\nResponsibilitiesAssist...,65000.0
...,...,...,...,...,...
13039,3906266217,The Dyrt,senior frontend app developer,web,NaN
13039,3906266217,The Dyrt,senior frontend app developer,API data mocking,NaN
13039,3906266217,The Dyrt,senior frontend app developer,and ORMCan do performance tuning of app and da...,NaN
13040,3906267126,Pinterest,staff software engineer ml serving platform,Improve the productivity and iteration speed o...,NaN


In [ ]:
skill_data.to_csv('filtered_it_skills.csv', index=False)